In [1]:
import subprocess
import downloader
from reader import Reader
from ontology import export_ontology

In [2]:
named_list = "fairness"

In [3]:
# downloader.download(named_list=named_list)

In [3]:
reader = Reader(named_list)
reader.load(from_inc=0)


Processing 10_1145-3442188_3445865.pdf: 100%|██████████| 201/201 [00:33<00:00,  6.00it/s]     


In [57]:
reader.extract_metadata()

Processing 10.1145/3442188.3445865: 100%|██████████| 31/31 [05:08<00:00,  9.97s/it]     


In [58]:
classifiers = ["acm", "dbpedia"]
reader.extract_classification(*classifiers)

Processing 10.1145/3442188.3445865: 100%|██████████| 31/31 [03:17<00:00,  6.37s/it]     


In [4]:
reader.get_metadata("dataframe")

,file_name,doi,title,authors,journal,publisher,year,keywords,topics_acm,topics_dbpedia
0,10_1145-3404835_3462948.pdf,10.1145/3404835.3462948,Fairness among New Items in Cold Start Recomme...,"Zhu, Ziwei;Kim, Jingu;Nguyen, Trung;Fenton, Ai...",,ACM,2021,fairness;cold start recommendation,information systems;information retrieval;retr...,rawlsian;recommender systems;generative model;...
1,10_1145-3303772_3303791.pdf,10.1145/3303772.3303791,Evaluating the Fairness of Predictive Student ...,"Gardner, Josh;Brooks, Christopher;Baker, Ryan",,ACM,2019,fairness;machine learning;moocs,applied computing;education;computer-assisted ...,learning analytics;metric;moocs;predictive mod...
2,10_1145-3616865.pdf,10.1145/3616865,Fairness in Machine Learning: A Survey,"Caton, Simon;Haas, Christian",ACM Computing Surveys,Association for Computing Machinery (ACM),2023,fairness;accountability;transparency;machine l...,NaN,
3,10_1145-3340531_3411980.pdf,10.1145/3340531.3411980,Fair Class Balancing: Enhancing Model Fairness...,"Yan, Shen;Kao, Hsien-te;Ferrara, Emilio",,ACM,2020,fairness;bias;class balancing,computing methodologies;machine learning;socia...,machine learning;screening;spur
4,10_1145-3359061_3361084.pdf,10.1145/3359061.3361084,Is mutation score a fair metric?,"Souza, Beatriz",,ACM,2019,mutation testing;mutation score;test suite eff...,software and its engineering;software creation...,metric;mutation;test suite
...,...,...,...,...,...,...,...,...,...,...
196,10_1145-3394486_3403199.pdf,10.1145/3394486.3403199,Evaluating Fairness Using Permutation Tests,"DiCiccio, Cyrus;Vasudevan, Sriram;Basu, Kinjal...",,ACM,2020,permutation tests;fairness;asymptotics,information systems;world wide web;web applica...,permutation;machine learning;gravity;metric;hy...
197,10_1145-3531146_3533146.pdf,10.1145/3531146.3533146,Fair Representation Clustering with Several Pr...,"Dai, Zhen;Makarychev, Yury;Vakilian, Ali",,ACM,2022,fair k-median;clustering;approximation algorit...,,k-median problem;metric space;ck;fj;exponentia...
198,10_1109-ICSE48619_2023_00133.pdf,10.1109/ICSE48619.2023.00133,,,,,,fairness;ensemble;machine learning;models,NaN,NaN
199,10_1145-3600211_3604657.pdf,10.1145/3600211.3604657,Fairness Implications of Encoding Protected Ca...,"Mougan, Carlos;Alvarez, Jose Manuel;Ruggieri, ...",,ACM,2023,fairness;algorithmic accountability;categorica...,computing methodologies;machine learning;learn...,


In [70]:
len([1 for paper in reader.paper_list if paper.topics.get("acm") or paper.topics.get("dbpedia")])

176

In [61]:
reader.dump(overwrite=False)

In [64]:
reader.export_as_klink_input(classification_source="acm")

,DE,TI,AU,SO,SC,PY
0,fairness;cold start recommendation,Fairness among New Items in Cold Start Recomme...,"Zhu, Ziwei;Kim, Jingu;Nguyen, Trung;Fenton, Ai...",ACM,information systems;information retrieval;retr...,2021
1,fairness;machine learning;moocs,Evaluating the Fairness of Predictive Student ...,"Gardner, Josh;Brooks, Christopher;Baker, Ryan",ACM,applied computing;education;computer-assisted ...,2019
3,fairness;bias;class balancing,Fair Class Balancing: Enhancing Model Fairness...,"Yan, Shen;Kao, Hsien-te;Ferrara, Emilio",ACM,computing methodologies;machine learning;socia...,2020
4,mutation testing;mutation score;test suite eff...,Is mutation score a fair metric?,"Souza, Beatriz",ACM,software and its engineering;software creation...,2019
5,fairness;spatial-temporal applications;privacy,Analysing Fairness of Privacy-Utility Mobility...,"Zhan, Yuting;Haddadi, Hamed;Mashhadi, Afra",ACM,security and privacy;human and societal aspect...,2023
...,...,...,...,...,...,...
192,fairness;named entity recognition;clinical de-...,In the Name of Fairness: Assessing the Bias in...,"Xiao, Yuxin;Lim, Shulammite;Pollard, Tom Josep...",ACM,computing methodologies;artificial intelligenc...,2023
194,decision trees;information gain;domain adaptat...,Domain Adaptive Decision Trees: Implications f...,"Alvarez, Jose M.;Scott, Kristen M.;Berendt, Be...",ACM,computing methodologies;machine learning;learn...,2023
196,permutation tests;fairness;asymptotics,Evaluating Fairness Using Permutation Tests,"DiCiccio, Cyrus;Vasudevan, Sriram;Basu, Kinjal...",ACM,information systems;world wide web;web applica...,2020
199,fairness;algorithmic accountability;categorica...,Fairness Implications of Encoding Protected Ca...,"Mougan, Carlos;Alvarez, Jose Manuel;Ruggieri, ...",ACM,computing methodologies;machine learning;learn...,2023


In [22]:

!cd "./klink2/"
subprocess.run(["Rscript", "main.R"])
!cd "../"

zsh:cd:1: no such file or directory: ./klink2/


process article  1  out of  1 
process keyword  1  out of  9 
process keyword  2  out of  9 
process keyword  3  out of  9 
process keyword  4  out of  9 
process keyword  5  out of  9 
process keyword  6  out of  9 
process keyword  7  out of  9 
process keyword  8  out of  9 
process keyword  9  out of  9 
Input variables saved to input-1.Rdata 



Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

hash-2.2.6.3 provided by Decision Patterns


Attaching package: ‘fastcluster’

The following object is masked from ‘package:stats’:

    hclust


Attaching package: ‘dplyr’

The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Iteration 1 
Number of keywords = 9 
Keyword inference.
Infering keyword: fairness  [ number of related keywords = 8 ]
Infering keyword: fcfs  [ number of related keywords = 8 ]
Infering keyword: job scheduling  [ number of related keywords = 8 ]
Infering keyword: m/m/1  [ number of related keywords = 8 ]
Infering keyword: processor sharing  [ number of related keywords = 8 ]
Infering keyword: ps  [ number of related keywords = 8 ]
Infering keyword: queue disciplines  [ number of related keywords = 8 ]
Infering keyword: resource allocation  [ number of related keywords = 8 ]
Infering keyword: unfairness  [ number of related keywords = 8 ]
Number of working semantic relations after inference:
	relatedEquivalent:  0 
	broaderGeneric:  0 
	contributesTo: 0 
	similarityLink: 0 
Seeking ambiguous keywords.
splitAmbiguousKeywords for fairness  [ number of related keywords = 8 ]
splitAmbiguousKeywords for fcfs  [ number of related keywords = 8 ]
splitAmbiguousKeywords for job scheduling  [ nu

In [3]:
export_ontology(named_list + "-1")

In [30]:
from reader import Paper
from config import PAPERS_PATH

In [31]:
paper = Paper(PAPERS_PATH / "fairness", "10_1145-3593013_3594107.pdf", True)

In [32]:
paper.load()

In [33]:
paper.abstract

'modern machine learning increasingly supports paradigms that are multi-institutional (using data from multiple institutions during training) or cross-institutional (using models from multiple institutions for inference), but the empirical effects of these paradigms are not well understood. this study investigates cross-institutional learning via an empirical case study in higher education. we propose a framework and metrics for assessing the utility and fairness of student dropout prediction models that are transferred across institutions. we examine the feasibility of cross-institutional transfer under real-world data- and model-sharing constraints, quantifying model biases for intersectional student identities, characterizing potential disparate impact due to these biases, and investigating the impact of various cross-institutional ensembling approaches on fairness and overall model performance. we perform this analysis on data representing over 200,000 enrolled students annually fr

In [35]:
paper._raw_text[0]

'Cross-Institutional Transfer Learning for Educational Models:\nImplications for Model Performance, Fairness, and Equity\nJosh Gardner\njpgard@cs.washington.edu\nUniversity of Washington\nRenzhe Yu\nrenzheyu@tc.columbia.edu\nTeachers College & Data Science\nInstitute, Columbia University\nQuan Nguyen\nquan.nguyen@ubc.ca\nUniversity of British Columbia\nChristopher Brooks\nbrooksch@umich.edu\nSchool of Information, University of\nMichigan\nRene F. Kizilcec\nkizilcec@cornell.edu\nDepartment of Information Science,\nCornell University\nABSTRACT\nModern machine learning increasingly supports paradigms that are\nmulti-institutional (using data from multiple institutions during\ntraining) or cross-institutional (using models from multiple insti-\ntutions for inference), but the empirical effects of these paradigms\nare not well understood. This study investigates cross-institutional\nlearning via an empirical case study in higher education. We pro-\npose a framework and metrics for assessing

In [49]:
from rdflib import Graph, URIRef, Literal, Namespace

# Create a Graph
g = Graph()

# Define a namespace
ex = Namespace("http://example.org/")

# Add triples
g.add((ex.Subject1, ex.hasProperty, ex.Object1))
g.add((ex.Subject2, ex.hasProperty, Literal("Literal Value")))

# Serialize to a file (Turtle format)
g.serialize(destination='ontology.ttl', format='turtle')

<Graph identifier=N0c46a3b59a004efd90d89db8c37da9f6 (<class 'rdflib.graph.Graph'>)>